In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel, Matern, RationalQuadratic, ExpSineSquared, DotProduct
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
import warnings
import pickle
import itertools
warnings.filterwarnings("ignore")


In [2]:
# Import data from 'Data/DATA_SOLMAT.xlsx'
doe = np.array(pd.read_excel('Data/DATA_SOLMAT.xlsx', engine='openpyxl',sheet_name = 'Normalized_table'))
data_exp = np.array(doe[:10])
X_exp = data_exp[:,:7]
Y_exp = data_exp[:,8:]

In [4]:
class GPUCB():

    def __init__(self, meshgrid, X, Y, beta=100.,delta = 0.1, dim = 2, pre_defined_gp = None):
        '''
        meshgrid: Output from np.methgrid.
        e.g. np.meshgrid(np.arange(-1, 1, 0.1), np.arange(-1, 1, 0.1)) for 2D space
        with |x_i| < 1 constraint.
        beta (optional): Hyper-parameter to tune the exploration-exploitation
        balance. If beta is large, it emphasizes the variance of the unexplored
        solution solution (i.e. larger curiosity)
        '''
        # 参考 https://github.com/tushuhei/gpucb/blob/master/gpucb.py        
        self.meshgrid = np.array(meshgrid)
        self.experimen_result= Y
        self.beta = beta
        self.iteration = 1
        self.dim = dim
        self.delta = delta
        self.acquired_pts = []
        self.x_only = []
        self.y_only = []  # for plotting
        if pre_defined_gp:
            self.gp = pre_defined_gp
        else:
            self.gp = GaussianProcessRegressor()
        self.X_grid = self.meshgrid.reshape(self.meshgrid.shape[0], -1).T
        self.mu, self.sigma = self.gp.predict(self.X_grid, return_std=True)
        self.X = []
        self.T = []
        
    def argmax_ucb(self):
        res = np.argmax(self.mu + self.sigma * np.sqrt(self.beta))
        return res

    def learn(self):
        grid_idx = self.argmax_ucb()
        self.sample(self.X_grid[grid_idx])
        self.beta = 2*np.log(self.X_grid.shape[0]*self.X_grid.shape[1]*(self.iteration**2)*(np.pi**2)/(6*self.delta))
        self.gp.fit(self.X, self.T)
        self.mu, self.sigma = self.gp.predict(self.X_grid, return_std=True)
        self.iteration += 1    
    
    def sample(self, x):
        print(x)
        t = self.experimen_result[tuple(x)]
        self.acquired_pts.append((x,t))
        self.y_only.append(t)
        self.x_only.append(x)
        self.X.append(x)
        self.T.append(t)

In [8]:
# temp 0-1 interval 0.2
# time interval 1/11
# pressure 1/18
# flow rate 1/9
# power 1/7
# distance 1/6
# ald_cycles 1/3
gp = GaussianProcessRegressor()
search_space = np.meshgrid(np.arange(0,1.01,0.2),np.arange(0,1.01,1/11),
                           np.arange(0,1.0001,1/18), np.arange(0,1.0001,1/9),
                           np.arange(0,1.0001,1/7), np.arange(0,1.0001,1/6),
                           np.arange(0,1.0001,1/3)
                          )
max_Y = np.max(Y_exp,axis=1)
gp.fit(X_exp,max_Y)
lookup = {tuple(X_exp[i]):Y_exp[i] for i in range(len(X_exp))}

test = GPUCB(search_space, X_exp, lookup, beta=0.1,delta = d, dim = 7, 
             pre_defined_gp = gp)
test.learn()

[0.6        0.45454545 0.33333333 0.44444444 0.42857143 0.33333333
 0.33333333]


KeyError: (0.6000000000000001, 0.4545454545454546, 0.3333333333333333, 0.4444444444444444, 0.42857142857142855, 0.3333333333333333, 0.3333333333333333)

In [9]:
np.arange(0,1.01,1/11)

array([0.        , 0.09090909, 0.18181818, 0.27272727, 0.36363636,
       0.45454545, 0.54545455, 0.63636364, 0.72727273, 0.81818182,
       0.90909091, 1.        ])

In [30]:
Y_exp

array([[13.267012271878247, 13.291169365031388, 13.097730015788807],
       [12.380161647939575, 12.579620556117561, 12.796080045330815],
       [12.687070145999757, 12.550012418398406, 12.41489666345802],
       [12.99989930059075, 13.02545506352055, 12.675656997387776],
       [12.87751270712218, 13.15255927203859, 13.290838306773587],
       [12.912010929748595, 12.961731079978332, 13.252564594879379],
       [13.139024889580085, 12.852233146014056, 13.06267547810554],
       [12.671878506909993, 12.540849550445612, 13.076600513568943],
       [11.893872765917578, 12.814372766025697, 12.850246345261953],
       [12.962852664182295, 13.02765378640283, 12.857221009916584]],
      dtype=object)